<a href="https://colab.research.google.com/github/daphnei/gen-ai-tutorials/blob/main/llm_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Imports and initialization

from abc import ABC

class Engine(ABC):
  def score(self, text):
    """Tokenizes the provided text, and returns a score for each token.
    
    The score is log-likelihood. A higher score means a token was more
    likely according to the model.
    """
    pass

  def generate(self, prompt, top_p, num_tokens):
    """Generates text given the provided prompt text."""
    pass


# Glossary

Here are some of the terms we will be using in this tutorial.


*   **(Language) Model**: A neural network trained to generate text.
*   **Engine**: The code and computers used to do inference with a model.
*   **Accelerator**: A [GPU](https://www.techtarget.com/searchvirtualdesktop/definition/GPU-graphics-processing-unit) or [TPU](https://cloud.google.com/tpu/docs/intro-to-tpu) hardware chip used to massively speed up model inference by enabling fast, parallelized matrix multiplcations. 



# Unit 1: Working with Pre-Trained Languages Models

## 1.1 Choosing a model and engine

Which model should you use? It depends on what your goals are, what your budget is, and what kinds of computational resources you have available.

In this section, we will summarize the pros and cons some of the popular systems, and guide you through the process of setting them up for inference.

## Open-source models on HuggingFace

HuggingFace is an open-source code framework for loading an open-source models onto an accelerator in order to train or do inference. It is very popular among academic researchers.

There are [hundreds of models](https://huggingface.co/models) available through HuggingFace's model repository. Some prominent ones which you can use for tutorial are.

### Why use this?
HuggingFace is great if your goal is to write open-source code, with results that anyone can reproduce. HuggingFace gives access to a huge number of models, and it is fairly easy to swap between models.

Because HuggingFace is widely used, it is easy to find help online. Probably someone else has had the same question as you and already posted about it.

### Why not use this?
When you use HuggingFace, you are running the code on your own computer. If you don't have a big enough accelerator, you will run into difficulties loading up larger models. For example, Colab (the software you are currently using) gives you access to a Tesla T4 GPU, which has 16 GB of RAM. This means, you can load the 6.7B parameter LLaMa, but not any of the bigger ones.

* **[Pythia-3B](https://huggingface.co/EleutherAI/pythia-2.8b-deduped)**: Part of a family of models trained by [Eleuther AI](https://www.eleuther.ai/about), a non-profit AI research lab.
* **[GPT-2 XL](https://huggingface.co/gpt2-xl)**: The original large language model from OpenAI, and the last one they open-sourced before moving away from open source models.
* **[BLOOM-3b](https://huggingface.co/docs/transformers/model_doc/bloom)**: These models were created by the [BigScience Initative](https://bigscience.huggingface.co/), a collaboration between HugginFace and many academic research labs to responsibly build a set of high-quality multilingual models.

## Open-source LLaMA 

Released this past February by Meta Research, LLaMA is the latest-and-greatest in open-source pre-trained LLMs. The LLaMA models range in size from 7 billion to 65 billion paramters.

Since the models' release, they have been finetuned by researchers to improve  functionality for a variety of use cases. Some prominent derivatives of LLaMA are:

* **Vicuna**: Finetuned for TODO
* **Alpaca**: Finetuned for TODO
* **TODO**:
 
While LLaMa and its derivative models can be used with the HuggingFace framework, the Colab runtime you are on right now doesn't have a big enough accelerator to load any of these models.

Instead, we have loaded up LLaMA on LTI's compute cluster, and you can use the code below to query the cluster. Note that the server being queries in the code below will be taken down at the end of the tutorial.

### Why use this?
TODO

### Why not use this?
TODO

### Setup Instructions
TODO

## Cohere's models

### Setup Instructions

1. Create an account [here](https://dashboard.cohere.ai/welcome/register).
2. Go to the [API key page](https://dashboard.cohere.ai/api-keys), and copy the `TRIAL` API key, then paste it into the code block below.
3. The `TRIAL` API Key allows you to make 5 API calls per-minute. If you would like to make more calls than that, go to the [API key page](https://dashboard.cohere.ai/api-keys) and press the "Get your production key" button. You will need to agree to their terms of service and provide credit card details in order to receieve a `PRODUCTION` API key.

In [1]:
!pip install cohere

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 11.1 MB/s eta 0:00:00


In [18]:
#@title Run this code block to use Cohere for Unit 1.
SECRET_KEY = "Paste your secret key here." #@param {type:"string"}

import cohere
co = cohere.Client(SECRET_KEY)

class CohereEngine(Engine):
  def __init__(self):
    pass

  def score(self, text):
    response = co.generate(
      prompt=text,
      num_generations=1,
      max_tokens=0,
      return_likelihoods="ALL"
    )
    return [(t.token, t.likelihood) for t in response[0].token_likelihoods]

  def generate(self, prompt, top_p, num_tokens):
    response = co.generate(
      prompt=prompt,
      num_generations=1,
      p=top_p,
      temperature=1.0,
      max_tokens=num_tokens,
      frequency_penalty=0.0,
      presence_penalty=0.0,
      return_likelihoods="ALL"
    )
    return response[0].text

engine = CohereEngine()

In [23]:
print(engine.generate("I visited Pittsburgh to", 0.5, 10))
print(engine.score("I visited Pittsburgh to see the CMU campus."))
print(engine.score("I visited Pittsburgh to see flying hippos."))


see the Cathedral of Learning at the University of
[('<BOS_TOKEN>', 0), ('I', -3.5298903), (' visited', -6.9353056), (' Pittsburgh', -8.640782), (' to', -3.871383), (' see', -1.9880424), (' the', -1.5376686), (' CMU', -9.146852), (' campus', -3.5548153), ('.', -2.5740197), ('<EOP_TOKEN>', -19.812128)]
[('<BOS_TOKEN>', 0), ('I', -3.5298903), (' visited', -6.9353056), (' Pittsburgh', -8.640782), (' to', -3.871383), (' see', -1.9880424), (' flying', -12.521135), (' hipp', -12.904638), ('os', -0.5945612), ('.', -1.4367169), ('<EOP_TOKEN>', -20.154535)]


In [17]:
[(t.token, t.likelihood) for t in r[0].token_likelihoods]

[('<BOS_TOKEN>', 0),
 ('When', -4.3959184),
 (' I', -2.375722),
 (' was', -1.2462932),
 (' five', -5.774685),
 (' years', -0.7006975),
 (' old', -0.048928205),
 (',', -0.38173616),
 (' I', -0.9145675),
 ('<EOP_TOKEN>', -20.68588)]

## OpenAI's GPT-3/4

TODO: Add brief summary of these models

### Why use these?
OpenAI's model have become the industry standard for large language models.
They have an API which is very easy to use. Since the models all get run on OpenAI's servers, you don't need your own compute resources, and there is practically no setup involved to get started.

### Why not use these?
OpenAI's models are not open-source and only accessible through an API. There is relatively little information available on how they were trained or what data they were trained on. OpenAI may change the models being used under the hood by the API, and you as a user will not know it. This makes them a bad choice for fully reproducible research.

The OpenAI API costs money to use. See [this link](https://openai.com/pricing) for pricing details.

### Setup Instructions
1. Go to www.openai.com and create an account.
2. Go to https://platform.openai.com/account/api-keys and click the "Create new secret key" button. It doesn't matter what you name it.
3. Copy the secret key and paste it into the code block below. Then run the code block.

In [24]:
#@title Run this code block to use GPT-3 for Unit 1.
MODEL_NAME = "text-davinci-003" #@param ["gpt-4", "text-davinci-003", "text-curie-001", "text-babbage-001", "text-ada-001"]
SECRET_KEY = "Paste your secret key here." #@param {type:"string"}

!pip install openai
import openai
openai.api_key = SECRET_KEY

class OpenAIEngine(Engine):
  def __init__(self, model_name):
    self.model_name = model_name

  def score(self, text):
    response = openai.Completion.create(
        engine=self.model_name,
        prompt=text,
        max_tokens=0,
        logprobs=1,
        echo=True)
    tokens = response["choices"][0]["logprobs"]["tokens"]
    scores = response["choices"][0]["logprobs"]["token_logprobs"]
    return list(zip(tokens, scores))

  def generate(self, prompt, top_p, num_tokens):
    response = openai.Completion.create(
      engine=self.model_name,
      prompt=prompt,
      temperature=1.0,
      max_tokens=num_tokens,
      top_p=top_p,
      frequency_penalty=0.0,
      presence_penalty=0.0,
      logprobs=1
    )
    return response["choices"][0]["text"]

engine = OpenAIEngine(MODEL_NAME)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.4 MB/s eta 0:00:00


In [36]:
# response = engine.score("I visited Pittsburgh to see the CMU campus.")
tokens = response["choices"][0]["logprobs"]["tokens"]
scores = response["choices"][0]["logprobs"]["token_logprobs"]
list(zip(tokens, scores))

[('I', None),
 (' visited', -9.353353),
 (' Pittsburgh', -8.062747),
 (' to', -4.9403515),
 (' see', -1.0997804),
 (' the', -0.9046383),
 (' CM', -9.227549),
 ('U', -0.0022087195),
 (' campus', -0.76313543),
 ('.', -3.3088434)]

en

## 1.3 Analyzing Likelihoods

As we discussed in the lecture, language models take as input a prompt sequence and then output a score for each token in the vocabulary.

A higher score means the model is more confident this that this token fits as the next token in the sequence.

In this unit, we will inspect token likelihoods to build an understaning of why models give tokens higher or low scores. We will also learn how to compute perplexity, a standard metric people use to measure how well models have learned to represent language.

In [ ]:
# TODO

## 1.3 Controlling the Amount of Randomness During Generation

In this section, we will investigate the impact the `top_p` parameter has on the text that a model generates.

In [ ]:
# TODO

## 1.4 Creating a Classifer using Few-Shot Learning

In this section, we will build a binary sentiment classifier using a LLM such
as GPT-3.

Writing the perfect prompt is more an art than a science, but a prompt often has
two parts:
- An `instruction` string that instructs the model on how to complete the task.
Adding it is often effective, because models such as OpenAI's `text-davinci-003`
are fine-tuned to follow user instructions.
- Sereral `demonstration` strings that give examples of completing the task.

For example, a prompt that translates English words to Chinese could look like
the following. Running GPT-3 to complete the prompt provides the correct answer
"松鼠".

```
Translate English to Chinese.

dog -> 狗
apple -> 苹果
coffee -> 咖啡
supermarket -> 超市
squirrel ->
```

Now, let's build a binary sentiment classifier for [yelp reviews](https://huggingface.co/datasets/yelp_polarity/).

In [ ]:
!pip install datasets

import datasets

yelp = datasets.load_dataset("yelp_polarity")
train_data = yelp["train"].shuffle(seed=1).select(range(4))
test_data = yelp["test"].shuffle(seed=1).select(range(20))

print("Data format:", train_data[1])


In [ ]:
# convert integer labels to text
label_map = {
    0: " negative",
    1: " positive"
}
"""
It might seem strange to put a space before "negative". This is due to how
GPT-3's tokenizer (BPE) works: " negative" is considered one token, and is
observed much more frequently than "negative" without the whitespace.
"""

instruction = "Classify the sentiment of these yelp reviews as positive or negative.\n\n"
demo_template = "Review: {review}\nSentiment:{sentiment}\n\n"

# construct the prompt by concatenating instructions and templates
prompt_parts = [instruction]
for instance in train_data:
    review = instance["text"]
    sentiment = label_map[instance["label"]]
    prompt_parts.append(demo_template.format(review=review, sentiment=sentiment))
prompt = ''.join(prompt_parts)


print("YOUR PROMPT:", prompt, sep='\n')

In [ ]:
# now we can feed the prompt to GPT-3 to classify a new review!

eval_template = "Review: {review}\nSentiment:"

def classify_review(review: str) -> str:
  """ Classify a single movie review """
  classify_prompt = prompt + eval_template.format(review=review)
  response = openai.Completion.create(
    engine=MODEL_NAME,
    prompt=classify_prompt,
    temperature=0.0, # <- only need the top-probability answer for classification
    max_tokens=1,
    logit_bias={
      "3967": 100,
      "4633": 100,
    } # <- whitelist only tokens that correspond to a label
  )
  return response["choices"][0]["text"]


def evaluate(verbose=False) -> None:
  """ Evaluate your prompt on the test set """
  correct = []
  for i, instance in enumerate(test_data):
    review = instance["text"]
    label = label_map[instance["label"]]
    predicted = classify_review(review)
    correct.append(1 if label == predicted else 0)
  
    print(f"======== {i} / {len(test_data)} ========")
    print(f"REVIEW: {review}")
    print(f"LABEL:{label}")
    print(f"PREDICTED:{predicted}")

  
  acc = sum(correct) / len(correct)
  print(f"Accuracy of your prompt on {len(test_data)} test examples: {acc:.0%}")


evaluate(verbose=True) # Expect 100% performance

# Unit 2: Models Tuned for Instruction-Following and Dialog

## 2.1 Choosing a model

## 2.2 Comparing Behaviour of Pre-trained and Tuned Models

## 2.3 Building a Persona Bot with In-Context Learning

# Scratch
This is where Daphne is keeping random code pieces which will not end up in the tutorial.

In [ ]:
engine.generate("What can", top_p=1.0, num_tokens=4)